# Create time-resolved transcriptome dicts

This workbook imports the "absolute" time-dependent transcriptome that was calculated by integrating the FISH experiments by Aouefa/Lotte with the RNAseq experiments by Lotte.

Results are pickled for use in the TRSL workflow.

04 August 2016: updated to include two extra genes by Wiebke

In [16]:
import pandas as pd
import cPickle as pkl
import math
import collections as col

In [17]:
transcript_file = '../data/FISH_RNAseq_combined_FACS_phases_7_genes.xlsx'

In [18]:
transcript_data = pd.read_excel(transcript_file, sheetname='nostress_WT_R1_rounded', 
                                skiprows=1, skipfooter=2, parse_cols='A, D:P')

Add systematic gene names:

In [19]:
from pygenome import sg

In [20]:
orf_genomic_dict = pkl.load(open("../parameters/orf_coding.p"))

systematic = []
for gene in transcript_data['Gene']:
    try:
        systematic.append(sg.systematic_name(gene))
    except:
        systematic.append(gene)
    
transcript_data['systematic'] = pd.Series(systematic)

In [21]:
transcript_data.head()

,Gene,0,5,10,15,20,25,30,35,40,45,50,55,60,systematic
0,YAL001C,0.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,0.0,1.0,1.0,YAL001C
1,YAL002W,1.0,0.0,1.0,1.0,1.0,2.0,2.0,3.0,1.0,1.0,0.0,1.0,1.0,YAL002W
2,YAL003W,17.0,13.0,48.0,51.0,51.0,100.0,100.0,104.0,27.0,47.0,23.0,46.0,31.0,YAL003W
3,YAL004W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,YAL004W
4,YAL005C,25.0,23.0,62.0,47.0,50.0,83.0,93.0,99.0,41.0,34.0,26.0,46.0,30.0,YAL005C


Just trying out how to select rows:

In [22]:
transcript_data[transcript_data['Gene'].isin(['YAL001C'])]

,Gene,0,5,10,15,20,25,30,35,40,45,50,55,60,systematic
0,YAL001C,0.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,0.0,1.0,1.0,YAL001C


In [23]:
transcriptome_time_dependent = {}

for t in range(0, 61, 5):
    transcriptome_time_dependent[t] = pd.Series([int(val) if not math.isnan(val) else 0 
                                                 for val in transcript_data[t].values],
                                                index=transcript_data['systematic']).to_dict()

In [24]:
print len(transcriptome_time_dependent[5])

6651


In [30]:
pkl.dump(transcriptome_time_dependent, open("../parameters/transcriptome_time_dependent.p", "wb"))

In [31]:
transcriptome_average = col.Counter()

for key in transcriptome_time_dependent:
    transcriptome_average = transcriptome_average + col.Counter(transcriptome_time_dependent[key])
    
transcriptome_average = {gene: transcriptome_average[gene] / len(transcriptome_time_dependent) 
                         for gene in transcriptome_average}

In [32]:
len(transcriptome_average)

5585

In [33]:
sum(transcriptome_average.values())

35812

In [34]:
pkl.dump(transcriptome_average, open("../parameters/transcriptome_teufel.p", "wb"))